In [1]:
import os
import azureml.core
import shutil

print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Azure ML SDK Version: 1.42.0


In [6]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/hoo-compute-cpu/code/Users/oli_hofmann/calorie_estimation2022/notebooks'

### Initialize Workspace configuration

In [15]:
DATASET_NAME = "UNIMIB2016-label_20220827_100542"
COMPUTE_NAME = "ML-Workspa-com1"
MODEL_NAME = "food-segmentation-model"

In [3]:
azure_ml_workspace = azureml.core.Workspace.from_config()

### Create experiment

In [4]:
mask_rcnn_experiment = azureml.core.Experiment(workspace=azure_ml_workspace, name="food-segmentation")

### Get compute target

In [13]:
compute_target = azure_ml_workspace.compute_targets[COMPUTE_NAME]

### Define Environment for the pipeline

In [7]:
pipeline_env = azureml.core.Environment.from_dockerfile("food-segmentation", 
                                                        dockerfile="../pipeline/Dockerfile",
                                                        pip_requirements="../pipeline/requirements.txt")

pipeline_env.register(workspace=azure_ml_workspace) 

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM tensorflow/tensorflow:2.5.0-gpu\n\n# Fix for the public key error\n# https://github.com/NVIDIA/nvidia-docker/issues/1632\nRUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub\nRUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/7fa2af80.pub\n\n\n# Install Common Dependencies\nRUN apt-get update && \\\n    apt-get install -y --no-install-recommends \\\n    wget\n\n# Conda Environment\nENV MINICONDA_VERSION py38_4.11.0\nENV PATH /opt/miniconda/bin:$PATH\nRUN wget -qO /tmp/miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-${MINICONDA_VERSION}-Linux-x86_64.sh && \\\n    bash /tmp/miniconda.sh -bf -p /opt/minicond

### Define dataset

In [11]:
dataset = azureml.core.Dataset.get_by_name(workspace=azure_ml_workspace, name=DATASET_NAME)
mount_context = dataset.mount(stream_column="image_url")

mount_context.start()
mount_context.mount_point

Method mount: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Not mounting as a volume: No datastore info was found, volume mount would not be attempted.. 
Falling back to dataflow mount.


'/tmp/tmpc0smxrnt'

In [9]:
df = dataset.to_pandas_dataframe()
df["image_url"] = df["image_url"].astype(str)

### Configure the training job

In [18]:
arguments = [
    "--dataset_name", DATASET_NAME,
    "--download_folder", "download",
    "--epochs", 5
    ]

script_run_config = azureml.core.ScriptRunConfig(
    source_directory="./..",
    script="train.py",
    compute_target=compute_target,
    environment=pipeline_env,
    arguments=arguments
)

### Submit the job to the cluster

In [19]:
run = mask_rcnn_experiment.submit(config=script_run_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
food-segmentation,food-segmentation_1661597154_752b264c,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation
